# Scrapping Youtube comments

In [15]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser

In [16]:
import pandas as pd
import json

In [17]:
DEVELOPER_KEY = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
YOUTUBE_API_SERVICE_NAME = "youtube"
YOUTUBE_API_VERSION = "v3"

In [19]:
def youtube_search(q, max_results=50,order="relevance", token=None, location=None, location_radius=None):

    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

    search_response = youtube.search().list(
        q=q,
        type="video",
        pageToken=token,
        order = order,
        part="id, snippet",
        maxResults=max_results,
        location=location,
        locationRadius=location_radius
    ).execute()

    

    videos = []

    for search_result in search_response.get("items", []):
        if search_result["id"]["kind"] == "youtube#video":
            videos.append(search_result)
    try:
        nexttok = search_response["nextPageToken"]
        return(nexttok, videos)
    except Exception as e:
        nexttok = "last_page"
        return(nexttok, videos)


def geo_query(video_id):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
                    developerKey=DEVELOPER_KEY)

    video_response = youtube.videos().list(
        id=video_id,
        part='snippet, recordingDetails, statistics'

    ).execute()

    return video_response

In [20]:
def build_service(filename):
    
    key = DEVELOPER_KEY

    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [21]:
# configure function parameters for required variables to pass to service
def get_comments(part='snippet', 
                 maxResults=100, 
                 searchTerms="Biden, Trump, election",
                 textFormat='plainText',
                 order='time',
                 videoId='yGPfKkjDIts',
                 station='',
                 debate='',
                 filename="plik.txt"):
    
    number_comment = 0

    service = build_service('../apikey.json')
    
    try:
        response = service.commentThreads().list(
            part=part,
            maxResults=maxResults,
            searchTerms=searchTerms,
            textFormat=textFormat,
            order=order,
            videoId=videoId
        ).execute()


        while response:

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                author = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
                published_time = item['snippet']['topLevelComment']['snippet']['publishedAt']
                reply_count = item['snippet']['totalReplyCount']
                like_count = item['snippet']['topLevelComment']['snippet']['likeCount']

                dates = [published_time, author, comment, reply_count, like_count, videoId, station, debate]

                try:
                    with open(filename, 'a') as f:
                        f.write("%s\n" % dates)
                        number_comment += 1
                except:
                    continue

            if 'nextPageToken' in response:
                response = service.commentThreads().list(
                    part=part,
                    maxResults=maxResults,
                    searchTerms=searchTerms,
                    textFormat=textFormat,
                    order=order,
                    videoId=videoId,
                    pageToken=response['nextPageToken']
                ).execute()
            else:
                break

    except:
        print(f'{videoId} has comments disabled, or something else went wrong')

    return number_comment

In [100]:
# Words: Trump, TRUMP BIDEN DEBATE, biden, election 2020, president 2020, elction results, debate 2020, election usa, win election 2020,kamala harris, mike pence,
# covid, health, inauguration,state union,united states,america,russia,china,Impeach,person,president usa
search = 'president trump'
searchTerms = 'Biden, Joe, Trump, Donald, election'

In [101]:
video_dict = {'youID':[], 'title':[], 'pub_date':[]}

In [102]:
def grab_videos(keyword, token=None):
    res = youtube_search(keyword, token=token)
    token = res[0]
    videos = res[1]
    for vid in videos:
        video_dict['youID'].append(vid['id']['videoId'])
        video_dict['title'].append(vid['snippet']['title'])
        video_dict['pub_date'].append(vid['snippet']['publishedAt'])
    print( "added " + str(len(videos)) + " videos to a total of " + str(len(video_dict['youID'])))
    return token

In [1]:
token = grab_videos(search)

while token != "last_page":
    token = grab_videos(search, token=token)

In [18]:
scrapped = []

In [315]:
kom = 0
for video in range(len(video_dict['youID'])):
    if video_dict['youID'][video] not in scrapped:
        kom += get_comments(videoId=video_dict['youID'][video], searchTerms=searchTerms, filename='komentarzeYT.txt', station=video_dict['title'][video], debate=video_dict['pub_date'][video])
        scrapped.append(video_dict['youID'][video])

print(f'Add {kom} comments')

zj0oLGVhrOc has comments disabled, or something else went wrong
wW1lY5jFNcQ has comments disabled, or something else went wrong
byBwWvgCDmU has comments disabled, or something else went wrong
HGAXBFYWKgM has comments disabled, or something else went wrong
JZk6VzSLe4Y has comments disabled, or something else went wrong
dfdtjPx6J1Y has comments disabled, or something else went wrong
_FLo14GMYos has comments disabled, or something else went wrong
dVxVDDYwNvU has comments disabled, or something else went wrong
j8k8hj5VyM0 has comments disabled, or something else went wrong
au3Zzf33vXw has comments disabled, or something else went wrong
DZqu8ojifhU has comments disabled, or something else went wrong
8ilyest1Z7U has comments disabled, or something else went wrong
dodano 86 komentarzy


In [73]:
len(scrapped)

3070

In [480]:
ytdict = {'published_time':[], 'author':[], 'comment':[], 'reply_count':[], 'like_count':[], 'videoId':[], 'videotitle':[], 'videotimepublished':[]}

In [481]:
z = open('komentarzeYT.csv',encoding='utf8')

In [482]:
d= z.readlines()

In [483]:
print(d[2].split())

['"2020-11-13T07:24:44Z"#k#"People""s', 'voice"#k#"US', 'elections', '2020', 'Joe', 'Biden', 'vs', 'Donald', 'Trump,https://www.youtube.com/watch?v=QWyrZ0-TZoo"#k#"0,', '0"#k#"UCA1A5GqCdQ"#k#"Final', '2020', 'Presidential', 'Debate', 'Between', 'Donald', 'Trump,', 'Joe', 'Biden', '|', 'NBC', 'News"#k#"2020-10-23T03:20:53Z"""']


In [484]:
string=[]
for k in range(0,len(d)):
    string.append(d[k])

In [485]:
len(d)

4426

In [486]:
for k in range(0,len(d)):
    ytdict['published_time'].append(d[k].split('#k#')[0].replace('"',''))
    ytdict['author'].append(d[k].split('#k#')[1].replace('"',''))
    ytdict['comment'].append(d[k].split('#k#')[2].replace('"',''))
    x = d[k].split('#k#')[3].replace('"','')
    ytdict['reply_count'].append(x.split()[0].replace(',',''))
    ytdict['like_count'].append(x.split()[1])
    ytdict['videoId'].append(d[k].split('#k#')[4].replace('"',''))
    ytdict['videotitle'].append(d[k].split('#k#')[5].replace('"',''))
    s = d[k].split('#k#')[6].replace('"','')
    ytdict['videotimepublished'].append(s.replace('\n',''))

In [487]:
dane = pd.DataFrame.from_dict(ytdict)

In [491]:
dane.videotimepublished = pd.to_datetime(dane.videotimepublished)

In [493]:
dane.to_pickle('YouTube_comments_DF.pkl')

In [494]:
yt = pd.read_pickle('YouTube_comments_DF.pkl')

In [495]:
yt

,published_time,author,comment,reply_count,like_count,videoId,videotitle,videotimepublished
0,2020-10-22 20:00:19+00:00,Si Tran,I’m standing for president Donald Trump . I ca...,0,0,K8Z9Kqhrh5c,Watch In Full: Trump versus Biden in the first...,2020-09-30 08:00:46+00:00
1,2020-10-14 15:30:52+00:00,Ian A,"Im trying to decide who to vote for, this make...",33,84,K8Z9Kqhrh5c,Watch In Full: Trump versus Biden in the first...,2020-09-30 08:00:46+00:00
2,2020-11-13 07:24:44+00:00,Peoples voice,"US elections 2020 Joe Biden vs Donald Trump,ht...",0,0,UCA1A5GqCdQ,Final 2020 Presidential Debate Between Donald ...,2020-10-23 03:20:53+00:00
3,2020-11-10 03:59:37+00:00,Peoples voice,US elections 2020 Joe Biden vs Donald Trump\nh...,0,0,UCA1A5GqCdQ,Final 2020 Presidential Debate Between Donald ...,2020-10-23 03:20:53+00:00
4,2020-11-06 16:50:01+00:00,Sean Simmons,Vote joe Biden Not Donald Trump,0,0,UCA1A5GqCdQ,Final 2020 Presidential Debate Between Donald ...,2020-10-23 03:20:53+00:00
...,...,...,...,...,...,...,...,...
4421,2020-11-07 19:21:48+00:00,Majestic,"Firstly, we need to understand: unsolicited ba...",0,0,KravPVeqTCc,Joe Biden defeats Donald Trump to become 46th ...,2020-11-07 16:55:00+00:00
4422,2020-11-07 18:15:59+00:00,Jay H,"Oh no, Fox is calling Joe Biden president elec...",0,0,KravPVeqTCc,Joe Biden defeats Donald Trump to become 46th ...,2020-11-07 16:55:00+00:00
4423,2020-11-07 18:05:39+00:00,Michael,We all know why Joe Biden is rushing to falsel...,0,0,KravPVeqTCc,Joe Biden defeats Donald Trump to become 46th ...,2020-11-07 16:55:00+00:00
4424,2020-11-07 17:20:35+00:00,Vesa Kaitera,"I is highly likely, that Joe Biden will win, b...",2,9,KravPVeqTCc,Joe Biden defeats Donald Trump to become 46th ...,2020-11-07 16:55:00+00:00
